# Nearest Neighbour

The purpose of this sectionn is to use nearest neighbour algorithm to predict house price. First, import and load source data.

In [41]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import datetime
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors, KNeighborsRegressor

path = './archive/'

kc_data = pd.read_csv(path + '/kc_house_data.csv')

Clean data. Remove obviously wrong record with 33 bedrooms, detected in "Outlier checking.ipynb". Then split data into train, test, final test data by 60/20/20 ratio.

In [42]:
data = kc_data.drop(['id','date'],axis = 1)
#remove obviously wrong record with 33 bedrooms, detected in "Outlier checking.ipynb"
data = data.drop(labels=15870, axis=0) 

y = data['price']
X = data.drop('price', axis=1)

# split the train, test and final test data
X_train, X_final_test, y_train, y_final_test = train_test_split(X, y, test_size=0.2, random_state=42) 
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.25, random_state=42) 


#-------------above: data clean and split--------------------------------------------

Then transform input data by standard normal distribution.

In [43]:
def norm(X):
        X_new = stats.zscore(X)
        X_mean = np.average(X, axis=0)
        X_sigma = np.var(X, axis=0)**0.5
        return X_new, X_mean, X_sigma

In [44]:
X_train_norm, X_train_mean, X_train_sigma = norm(X_train)
X_test_norm, X_test_mean, X_test_sigma = norm(X_test)


Then fit to nearest neighbour model. Select KD tree algorithm for the model as it is good for large number of samples with dimension < 20.

In [53]:
for k in range(2,21):
    neigh = KNeighborsRegressor(n_neighbors=k, algorithm='kd_tree')
    neigh.fit(X_train_norm, y_train)
    print("R2 for k="+str(k)+": "+str(neigh.score(X_test_norm, y_test)))

R2 for k=2: 0.7580080642589794
R2 for k=3: 0.7753317790865841
R2 for k=4: 0.7772805172795035
R2 for k=5: 0.7795239500146574
R2 for k=6: 0.7774976447676919
R2 for k=7: 0.7794629035603614
R2 for k=8: 0.7785722564094976
R2 for k=9: 0.7776880808317788
R2 for k=10: 0.7784694908705883
R2 for k=11: 0.7794300000386134
R2 for k=12: 0.7773589320728365
R2 for k=13: 0.7763503241862784
R2 for k=14: 0.7768511497291425
R2 for k=15: 0.7759539902993505
R2 for k=16: 0.7729336681572083
R2 for k=17: 0.7729253510506396
R2 for k=18: 0.7705550898983238
R2 for k=19: 0.7697482091319399
R2 for k=20: 0.7690066610251997
